In [1]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = 140-96*x+12*x*x
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 100
    child_num = 50

    space = OrderedDict((
        ('x', (-2., 150.)),
        ('y', (0., 0.)),
        ('z', (0., 0.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 3.8346727159642944, 0.0, 0.0: -51.67200266984051
Eval 114.70907104993293, 0.0, 0.0: 147025.98095286914
Eval 31.42133425605718, 0.0, 0.0: 8971.15486858898
Eval 8.741213090512687, 0.0, 0.0: 217.74921883578645
Eval 142.85198203763363, 0.0, 0.0: 231306.474989352
Eval 17.951774729054556, 0.0, 0.0: 2283.8242170834237
Eval 86.50190285585373, 0.0, 0.0: 81626.76769804071
Eval 37.544783776585746, 0.0, 0.0: 13451.030223414684
Eval 36.67689143097698, 0.0, 0.0: 12761.350803102281
Eval 80.97409198438186, 0.0, 0.0: 71048.13004184095
Eval 98.9498378074595, 0.0, 0.0: 108133.66039595439
Eval 65.92155358619974, 0.0, 0.0: 45959.34558234327
Eval 107.47247410892192, 0.0, 0.0: 128426.63477865819
Eval 90.64996690840421, 0.0, 0.0: 90046.60118273053
Eval 104.88845376290598, 0.0, 0.0: 122089.7612320402
Eval 81.54963156118879, 0.0, 0.0: 72115.34426331354
Eval 29.68566370282179, 0.0, 0.0: 7865.03983825348
Eval 24.17153782482052, 0.0, 0.0: 4830.691258617979
Eval 29.191302347619757, 0.0, 0.0: 7563.220567630313
